In [1]:
!pip install 'git+https://github.com/katsura-jp/pytorch-cosine-annealing-with-warmup'
!pip install 'git+https://github.com/seungjunlee96/DepthwiseSeparableConvolution_Pytorch.git'
!pip install -U fvcore

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/katsura-jp/pytorch-cosine-annealing-with-warmup to /tmp/pip-req-build-3s7_lr01
  Running command git clone --filter=blob:none --quiet https://github.com/katsura-jp/pytorch-cosine-annealing-with-warmup /tmp/pip-req-build-3s7_lr01
  Resolved https://github.com/katsura-jp/pytorch-cosine-annealing-with-warmup to commit 12d03c07553aedd3d9e9155e2b3e31ce8c64081a
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/seungjunlee96/DepthwiseSeparableConvolution_Pytorch.git to /tmp/pip-req-build-pr8h0nne
  Running command git clone --filter=blob:none --quiet https://github.com/seungjunlee96/DepthwiseSeparableConvolution_Pytorch.git /tmp/pip-req-build-pr8h0nne
  Resolved https://github.com/seungjunlee96/DepthwiseSeparableConvolution_Pytorch.git to commit b16d

In [2]:
import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet101

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from cosine_annealing_warmup import CosineAnnealingWarmupRestarts

import torch.nn.utils.prune as prune

from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

import pathlib
import os
import copy

In [3]:
def evaluate_model(model, loss_fn, device, test_loader):
    model.eval()
    model.to(device)

    _loss = 0
    _corrects = 0
    total = 0

    with torch.no_grad():
        for i_batch, (inputs, labels) in enumerate(test_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
          
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
              
            _loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            _corrects += (predicted == labels).sum().item()   

        _test_loss = _loss
        _test_acc = _corrects/total
        _best_acc = _test_acc

    return _test_loss, _test_acc

In [51]:
def measure_module_sparsity(module, weight=True, bias=False, use_mask=False):

    num_zeros = 0
    num_elements = 0

    if use_mask == True:
        for buffer_name, buffer in module.named_buffers():
            if "weight_mask" in buffer_name and weight == True:
                num_zeros += torch.sum(buffer == 0).item()
                num_elements += buffer.nelement()
            if "bias_mask" in buffer_name and bias == True:
                num_zeros += torch.sum(buffer == 0).item()
                num_elements += buffer.nelement()
    else:
        for param_name, param in module.named_parameters():
            if "weight" in param_name and weight == True:
                num_zeros += torch.sum(param == 0).item()
                num_elements += param.nelement()
            if "bias" in param_name and bias == True:
                num_zeros += torch.sum(param == 0).item()
                num_elements += param.nelement()
    sparsity = num_zeros / num_elements

    return num_zeros, num_elements, sparsity

In [52]:
def measure_global_sparsity(
    model, weight = True,
    bias = False, conv2d_use_mask = False,
    linear_use_mask = False):

    num_zeros = 0
    num_elements = 0

    for module_name, module in model.named_modules():

        if isinstance(module, torch.nn.Conv2d):

            module_num_zeros, module_num_elements, _ = measure_module_sparsity(
                module, weight=weight, bias=bias, use_mask=conv2d_use_mask)
            num_zeros += module_num_zeros
            num_elements += module_num_elements

        elif isinstance(module, torch.nn.Linear):

            module_num_zeros, module_num_elements, _ = measure_module_sparsity(
                module, weight=weight, bias=bias, use_mask=linear_use_mask)
            num_zeros += module_num_zeros
            num_elements += module_num_elements

    sparsity = num_zeros / num_elements

    return num_zeros, num_elements, sparsity

In [6]:
def fine_tune_model(model, train_loader, test_loader, device, loss_fn, optimizer, num_epochs):
    model.to(device)

    

    scheduler = CosineAnnealingWarmupRestarts(optimizer,
                                          first_cycle_steps=200,
                                          cycle_mult=1.0,
                                          max_lr=0.1,
                                          min_lr=0.0001,
                                          warmup_steps=int(num_epochs/4),
                                          gamma=1.0)

    for epoch in range(num_epochs):
        _loss = 0.0
        model.train()
        for i_batch, (inputs, labels) in enumerate(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward(retain_graph=True)
            optimizer.step()

            _loss += loss.item()

        _train_loss = _loss

        _test_loss, _test_acc = evaluate_model(model, loss_fn, device, test_loader)


        print(f"epoch = {epoch} train loss = {_train_loss:.3f}, test_loss = {_test_loss:.3f}, test_accuracy = {_test_acc:.4f}")
        scheduler.step()


    return model

In [7]:
def remove_parameters(model):

    for module_name, module in model.named_modules():
        if isinstance(module, torch.nn.Conv2d):
            try:
                prune.remove(module, "weight")
            except:
                pass
            try:
                prune.remove(module, "bias")
            except:
                pass
        elif isinstance(module, torch.nn.Linear):
            try:
                prune.remove(module, "weight")
            except:
                pass
            try:
                prune.remove(module, "bias")
            except:
                pass

    return model

In [8]:
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

In [30]:
from prettytable import PrettyTable
def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

In [9]:
batch_size = 256

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [10]:
transform_train = transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomRotation(20),
    transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data/',
                                             train=True, 
                                             transform=transform_train,
                                             download=True)

train_loader = torch.utils.data.DataLoader(train_dataset, 
                                          batch_size=batch_size,
                                          shuffle=True, 
                                          num_workers=2)

test_dataset = torchvision.datasets.CIFAR10(root='./data', 
                                       train=False,
                                       download=True, 
                                       transform=transform_test)

test_loader = torch.utils.data.DataLoader(test_dataset,
                                         batch_size=batch_size,
                                         shuffle=False, 
                                         num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [11]:
dataiter = iter(test_loader)
imgs, labels = next(dataiter)
# imgs[0]

In [12]:
import math

expansion_ratio = 32/192

widths = [64, 128, 256, 512]
input_channels = []

for w in widths:
    x = w * expansion_ratio
    if math.floor(x) % 2 == 0:
        input_channels.append(math.floor(x))
    else:
        input_channels.append(math.floor(x) + 1)
    
for x in input_channels:
    print(x)

10
22
42
86


In [20]:
class depthwise_separable_conv(nn.Module):
    def __init__(self, nin, nout, kernel_size = 3, padding = 1, stride = 1, bias=False):
        super(depthwise_separable_conv, self).__init__()
        self.depthwise = nn.Conv2d(nin, nin, kernel_size=kernel_size, padding=padding, stride=(stride, stride) , groups=nin, bias=bias)
        self.pointwise = nn.Conv2d(nin, nout, kernel_size=1, bias=bias)

    def forward(self, x):
        out = self.depthwise(x)
        out = self.pointwise(out)
        return out


class invertedResidual(nn.Module):
    def __init__(self, nin, nhidden, nout, stride = 1, downsample=False):
        super(invertedResidual, self).__init__()

        self.nin = nin
        self.nhidden = nhidden
        self.nout = nout
        self.downsample = downsample

        # self.ff = torch.nn.quantized.FloatFunctional()

        self.quant = torch.quantization.QuantStub()
        self.dequant = torch.quantization.DeQuantStub()

        if self.downsample:
            self.downsampleLayer = nn.Sequential(
                nn.Conv2d(nin, nout, kernel_size=(1, 1), stride=stride, bias=False),
                nn.BatchNorm2d(nout)
            )
        else:
            self.downsampleLayer = None

        self.conv1 = nn.Conv2d(nin, nhidden, kernel_size=1, stride=1, bias=False)
        self.bn1 = nn.BatchNorm2d(nhidden)
        self.relu1 = nn.ReLU6(inplace=True)

        self.conv2 = depthwise_separable_conv(nhidden, nhidden, kernel_size = 3, stride=stride, padding = 1, bias=False)
        self.bn2 = nn.BatchNorm2d(nhidden)
        self.relu2 = nn.ReLU6(inplace=True)

        if nhidden != nout:
            self.conv3 = nn.Conv2d(nhidden, nout, kernel_size=1, stride=1, bias=False)
            self.bn3 = nn.BatchNorm2d(nout)



    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu2(out)

        if self.nhidden != self.nout:
            out = self.conv3(out)
            out = self.bn3(out)

        if self.downsample:
            identity = self.downsampleLayer(x)

        identity = self.dequant(identity)
        out = self.dequant(out)
        out += identity
        out = self.quant(out)

        return out


model = resnet101(pretrained=False, num_classes=10)

model.conv1 = depthwise_separable_conv(3, 64, kernel_size = 3, padding = 1, bias=False)

model.maxpool = nn.Identity()

#Change to inverted residual blocks with depthwise convolutional layer


#layer1
model.layer1 = nn.Sequential(
    invertedResidual(64, 64, 10, stride=1, downsample=True),
    invertedResidual(input_channels[0], widths[0], input_channels[0]),
    invertedResidual(input_channels[0], widths[0], input_channels[0]),
    invertedResidual(input_channels[0], widths[0], input_channels[1], stride=2, downsample=True)
)
#layer2
model.layer2[0]= invertedResidual(input_channels[1], widths[1], input_channels[1])
model.layer2[1] = invertedResidual(input_channels[1], widths[1], input_channels[1])
model.layer2[2] = invertedResidual(input_channels[1], widths[1], input_channels[1])
model.layer2[3] = invertedResidual(input_channels[1], widths[1], input_channels[2], stride=2, downsample=True)
#layer3
model.layer3[0] = invertedResidual(input_channels[2], widths[2], input_channels[2])
model.layer3[1] = invertedResidual(input_channels[2], widths[2], input_channels[2])
model.layer3[2] = invertedResidual(input_channels[2], widths[2], input_channels[2])
model.layer3[3] = invertedResidual(input_channels[2], widths[2], input_channels[2])
model.layer3[4] = invertedResidual(input_channels[2], widths[2], input_channels[2])
model.layer3[5] = invertedResidual(input_channels[2], widths[2], input_channels[2])
model.layer3[6] = invertedResidual(input_channels[2], widths[2], input_channels[2])
model.layer3[7] = invertedResidual(input_channels[2], widths[2], input_channels[2])
model.layer3[8] = invertedResidual(input_channels[2], widths[2], input_channels[2])
model.layer3[9] = invertedResidual(input_channels[2], widths[2], input_channels[2])
model.layer3[10] = invertedResidual(input_channels[2], widths[2], input_channels[2])
model.layer3[11] = invertedResidual(input_channels[2], widths[2], input_channels[2])
model.layer3[12] = invertedResidual(input_channels[2], widths[2], input_channels[2])
model.layer3[13] = invertedResidual(input_channels[2], widths[2], input_channels[2])
model.layer3[14] = invertedResidual(input_channels[2], widths[2], input_channels[2])
model.layer3[15] = invertedResidual(input_channels[2], widths[2], input_channels[2])
model.layer3[16] = invertedResidual(input_channels[2], widths[2], input_channels[2])
model.layer3[17] = invertedResidual(input_channels[2], widths[2], input_channels[2])
model.layer3[18] = invertedResidual(input_channels[2], widths[2], input_channels[2])
model.layer3[19] = invertedResidual(input_channels[2], widths[2], input_channels[2])
model.layer3[20] = invertedResidual(input_channels[2], widths[2], input_channels[2])
model.layer3[21] = invertedResidual(input_channels[2], widths[2], input_channels[2])
model.layer3[22] = invertedResidual(input_channels[2], widths[2], input_channels[3], stride=2, downsample=True)
#layer4
model.layer4 = nn.Sequential(
    invertedResidual(input_channels[3], widths[3], input_channels[3]),
    invertedResidual(input_channels[3], widths[3], input_channels[3]),
    nn.Conv2d(input_channels[3], widths[3], kernel_size=1, stride=1, bias=False),
    nn.BatchNorm2d(widths[3]),
    nn.ReLU6(inplace=True)
)

model.fc = nn.Sequential(
    nn.Linear(widths[3], widths[3], bias=False),
    nn.ReLU6(inplace=True),
    nn.Linear(widths[3], 10, bias=False),
)

model.to(device)
model.train()

ResNet(
  (conv1): depthwise_separable_conv(
    (depthwise): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=3, bias=False)
    (pointwise): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): Identity()
  (layer1): Sequential(
    (0): invertedResidual(
      (quant): QuantStub()
      (dequant): DeQuantStub()
      (downsampleLayer): Sequential(
        (0): Conv2d(64, 10, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU6(inplace=True)
      (conv2): depthwise_separable_conv(
        (depthwise): Conv2d(64, 64, kernel_size=(3, 3), stri

In [12]:
summary(model.cuda(), (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 3, 32, 32]              27
            Conv2d-2           [-1, 64, 32, 32]             192
depthwise_separable_conv-3           [-1, 64, 32, 32]               0
       BatchNorm2d-4           [-1, 64, 32, 32]             128
              ReLU-5           [-1, 64, 32, 32]               0
          Identity-6           [-1, 64, 32, 32]               0
            Conv2d-7           [-1, 64, 32, 32]           4,096
       BatchNorm2d-8           [-1, 64, 32, 32]             128
             ReLU6-9           [-1, 64, 32, 32]               0
           Conv2d-10           [-1, 64, 32, 32]             576
           Conv2d-11           [-1, 64, 32, 32]           4,096
depthwise_separable_conv-12           [-1, 64, 32, 32]               0
      BatchNorm2d-13           [-1, 64, 32, 32]             128
            ReLU6-14      

In [21]:
from google.colab import drive
drive.mount('/content/drive')

save_dir = pathlib.Path('/content/drive/MyDrive/Colab_Notebooks/MGU/Resnet_CIFAR10_DSC_InvertedRes_GAP_output')

model_name = 'resnet101.151.h5'

model.load_state_dict(torch.load(os.path.join(save_dir, model_name)))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<All keys matched successfully>

In [23]:
pruned_model = copy.deepcopy(model)

In [15]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-5)

In [26]:
test_loss, test_acc = evaluate_model(model, loss_fn, 'cpu', test_loader)

print(f"Accuracy = {test_acc:.4f} & Loss = {test_loss:.4f}")

Accuracy = 0.9280 & Loss = 17.5915


In [16]:
class QuantizedResNet(nn.Module):
    def __init__(self, model):
        super(QuantizedResNet, self).__init__()

        self.quant = torch.quantization.QuantStub()
        self.dequant = torch.quantization.DeQuantStub()
        self.model = model

    def forward(self, x):
        x = self.quant(x)
        x = self.model(x)
        x = self.dequant(x)
        return x

model_to_quant = QuantizedResNet(model)

# Quantization

In [19]:
modules_to_fuse = [
    ['model.bn1', 'model.relu'],

    #layer1
    ['model.layer1.0.downsampleLayer.0', 'model.layer1.0.downsampleLayer.1'],
    ['model.layer1.0.conv1', 'model.layer1.0.bn1'],
    ['model.layer1.0.conv2.pointwise', 'model.layer1.0.bn2'],
    ['model.layer1.0.conv3', 'model.layer1.0.bn3'],

    ['model.layer1.1.conv1', 'model.layer1.1.bn1'],
    ['model.layer1.1.conv2.pointwise', 'model.layer1.1.bn2'],
    ['model.layer1.1.conv3', 'model.layer1.1.bn3'],

    ['model.layer1.2.conv1', 'model.layer1.2.bn1'],
    ['model.layer1.2.conv2.pointwise', 'model.layer1.2.bn2'],
    ['model.layer1.2.conv3', 'model.layer1.2.bn3'],

    ['model.layer1.3.downsampleLayer.0', 'model.layer1.3.downsampleLayer.1'],
    ['model.layer1.3.conv1', 'model.layer1.3.bn1'],
    ['model.layer1.3.conv2.pointwise', 'model.layer1.3.bn2'],
    ['model.layer1.3.conv3', 'model.layer1.3.bn3'],

    #layer2
    ['model.layer2.0.conv1', 'model.layer2.0.bn1'],
    ['model.layer2.0.conv2.pointwise', 'model.layer2.0.bn2'],
    ['model.layer2.0.conv3', 'model.layer2.0.bn3'],

    ['model.layer2.1.conv1', 'model.layer2.1.bn1'],
    ['model.layer2.1.conv2.pointwise', 'model.layer2.1.bn2'],
    ['model.layer2.1.conv3', 'model.layer2.1.bn3'],

    ['model.layer2.2.conv1', 'model.layer2.2.bn1'],
    ['model.layer2.2.conv2.pointwise', 'model.layer2.2.bn2'],
    ['model.layer2.2.conv3', 'model.layer2.2.bn3'],

    ['model.layer2.3.downsampleLayer.0', 'model.layer2.3.downsampleLayer.1'],
    ['model.layer2.3.conv1', 'model.layer2.3.bn1'],
    ['model.layer2.3.conv2.pointwise', 'model.layer2.3.bn2'],
    ['model.layer2.3.conv3', 'model.layer2.3.bn3'],

    #layer3
    ['model.layer3.0.conv1', 'model.layer3.0.bn1'],
    ['model.layer3.0.conv2.pointwise', 'model.layer3.0.bn2'],
    ['model.layer3.0.conv3', 'model.layer3.0.bn3'],

    ['model.layer3.1.conv1', 'model.layer3.1.bn1'],
    ['model.layer3.1.conv2.pointwise', 'model.layer3.1.bn2'],
    ['model.layer3.1.conv3', 'model.layer3.1.bn3'],

    ['model.layer3.2.conv1', 'model.layer3.2.bn1'],
    ['model.layer3.2.conv2.pointwise', 'model.layer3.2.bn2'],
    ['model.layer3.2.conv3', 'model.layer3.2.bn3'],

    ['model.layer3.3.conv1', 'model.layer3.3.bn1'],
    ['model.layer3.3.conv2.pointwise', 'model.layer3.3.bn2'],
    ['model.layer3.3.conv3', 'model.layer3.3.bn3'],

    ['model.layer3.4.conv1', 'model.layer3.4.bn1'],
    ['model.layer3.4.conv2.pointwise', 'model.layer3.4.bn2'],
    ['model.layer3.4.conv3', 'model.layer3.4.bn3'],

    ['model.layer3.5.conv1', 'model.layer3.5.bn1'],
    ['model.layer3.5.conv2.pointwise', 'model.layer3.5.bn2'],
    ['model.layer3.5.conv3', 'model.layer3.5.bn3'],

    ['model.layer3.6.conv1', 'model.layer3.6.bn1'],
    ['model.layer3.6.conv2.pointwise', 'model.layer3.6.bn2'],
    ['model.layer3.6.conv3', 'model.layer3.6.bn3'],

    ['model.layer3.7.conv1', 'model.layer3.7.bn1'],
    ['model.layer3.7.conv2.pointwise', 'model.layer3.7.bn2'],
    ['model.layer3.7.conv3', 'model.layer3.7.bn3'],

    ['model.layer3.8.conv1', 'model.layer3.8.bn1'],
    ['model.layer3.8.conv2.pointwise', 'model.layer3.8.bn2'],
    ['model.layer3.8.conv3', 'model.layer3.8.bn3'],

    ['model.layer3.9.conv1', 'model.layer3.9.bn1'],
    ['model.layer3.9.conv2.pointwise', 'model.layer3.9.bn2'],
    ['model.layer3.9.conv3', 'model.layer3.9.bn3'],

    ['model.layer3.10.conv1', 'model.layer3.10.bn1'],
    ['model.layer3.10.conv2.pointwise', 'model.layer3.10.bn2'],
    ['model.layer3.10.conv3', 'model.layer3.10.bn3'],

    ['model.layer3.11.conv1', 'model.layer3.11.bn1'],
    ['model.layer3.11.conv2.pointwise', 'model.layer3.11.bn2'],
    ['model.layer3.11.conv3', 'model.layer3.11.bn3'],

    ['model.layer3.12.conv1', 'model.layer3.12.bn1'],
    ['model.layer3.12.conv2.pointwise', 'model.layer3.12.bn2'],
    ['model.layer3.12.conv3', 'model.layer3.12.bn3'],

    ['model.layer3.13.conv1', 'model.layer3.13.bn1'],
    ['model.layer3.13.conv2.pointwise', 'model.layer3.13.bn2'],
    ['model.layer3.13.conv3', 'model.layer3.13.bn3'],

    ['model.layer3.14.conv1', 'model.layer3.14.bn1'],
    ['model.layer3.14.conv2.pointwise', 'model.layer3.14.bn2'],
    ['model.layer3.14.conv3', 'model.layer3.14.bn3'],

    ['model.layer3.15.conv1', 'model.layer3.15.bn1'],
    ['model.layer3.15.conv2.pointwise', 'model.layer3.15.bn2'],
    ['model.layer3.15.conv3', 'model.layer3.15.bn3'],

    ['model.layer3.16.conv1', 'model.layer3.16.bn1'],
    ['model.layer3.16.conv2.pointwise', 'model.layer3.16.bn2'],
    ['model.layer3.16.conv3', 'model.layer3.16.bn3'],

    ['model.layer3.17.conv1', 'model.layer3.17.bn1'],
    ['model.layer3.17.conv2.pointwise', 'model.layer3.17.bn2'],
    ['model.layer3.17.conv3', 'model.layer3.17.bn3'],

    ['model.layer3.18.conv1', 'model.layer3.18.bn1'],
    ['model.layer3.18.conv2.pointwise', 'model.layer3.18.bn2'],
    ['model.layer3.18.conv3', 'model.layer3.18.bn3'],

    ['model.layer3.19.conv1', 'model.layer3.19.bn1'],
    ['model.layer3.19.conv2.pointwise', 'model.layer3.19.bn2'],
    ['model.layer3.19.conv3', 'model.layer3.19.bn3'],

    ['model.layer3.20.conv1', 'model.layer3.20.bn1'],
    ['model.layer3.20.conv2.pointwise', 'model.layer3.20.bn2'],
    ['model.layer3.20.conv3', 'model.layer3.20.bn3'],

    ['model.layer3.21.conv1', 'model.layer3.21.bn1'],
    ['model.layer3.21.conv2.pointwise', 'model.layer3.21.bn2'],
    ['model.layer3.21.conv3', 'model.layer3.21.bn3'],

    ['model.layer3.22.conv1', 'model.layer3.22.bn1'],
    ['model.layer3.22.conv2.pointwise', 'model.layer3.22.bn2'],
    ['model.layer3.22.conv3', 'model.layer3.22.bn3'],

    #layer4
    ['model.layer4.0.conv1', 'model.layer4.0.bn1'],
    ['model.layer4.0.conv2.pointwise', 'model.layer4.0.bn2'],
    ['model.layer4.0.conv3', 'model.layer4.0.bn3'],

    ['model.layer4.1.conv1', 'model.layer4.1.bn1'],
    ['model.layer4.1.conv2.pointwise', 'model.layer4.1.bn2'],
    ['model.layer4.1.conv3', 'model.layer4.1.bn3'],

    ['model.layer4.2', 'model.layer4.3'],
]



In [20]:
def evaluate(model, loss_fn, data_loader, neval_batches):
    model.eval()
    cnt = 0
    with torch.no_grad():
        for image, target in data_loader:
            output = model(image)
            loss = loss_fn(output, target)
            cnt += 1
            test_loss, test_acc = evaluate_model(model, loss_fn, 'cpu', test_loader)
            print(f"Batch num = {cnt} & Accuracy = {test_acc:.4f} & Loss = {test_loss:.4f}")
            if cnt >= neval_batches:
                 return test_loss, test_acc

    return test_loss, test_acc

In [21]:
num_calibration_batches = 1

model_to_quant.to('cpu')
model_to_quant.eval()

quantization_config = torch.quantization.get_default_qconfig("qnnpack")
model_to_quant.qconfig = quantization_config

model_to_quant_fused = torch.quantization.fuse_modules(model_to_quant, modules_to_fuse)

model_to_quant_prepared = torch.quantization.prepare(model_to_quant_fused, inplace=False)

test_loss, test_acc = evaluate(model_to_quant_prepared, loss_fn, test_loader, neval_batches=num_calibration_batches)
print(f"Tunned: Accuracy = {test_acc:.4f} & Loss = {test_loss:.4f}")

print(model_to_quant_prepared.model.layer1[0].conv1)
model_quantized = torch.quantization.convert(model_to_quant_prepared, inplace=False)

Batch num = 1 & Accuracy = 0.9280 & Loss = 17.5915
Tunned: Accuracy = 0.9280 & Loss = 17.5915
Conv2d(
  64, 64, kernel_size=(1, 1), stride=(1, 1)
  (activation_post_process): HistogramObserver(min_val=-7.275644302368164, max_val=8.180307388305664)
)


In [54]:
model_name = 'quantized_resnet101.h5'
save_dir = pathlib.Path('/content/drive/MyDrive/Colab_Notebooks/MGU/Resnet_CIFAR10_Quantized')

torch.save(model_quantized.state_dict(), os.path.join(save_dir, model_name))

In [65]:
import torch.quantization._numeric_suite as ns

def compute_error(x, y):
    Ps = torch.norm(x)
    Pn = torch.norm(x-y)
    return 20*torch.log10(Ps/Pn)


wt_compare_dict = ns.compare_weights(model_to_quant.state_dict(), model_quantized.state_dict())
for key in wt_compare_dict:
    print(key, compute_error(wt_compare_dict[key]['float'], wt_compare_dict[key]['quantized'].dequantize()))


model.conv1.depthwise.weight tensor(42.8360)
model.conv1.pointwise.weight tensor(45.4614)
model.bn1.weight tensor(inf)
model.layer1.0.downsampleLayer.0.weight tensor(3.9769)
model.layer1.0.conv1.weight tensor(6.0337)
model.layer1.0.conv2.depthwise.weight tensor(39.9796)
model.layer1.0.conv2.pointwise.weight tensor(10.3279)
model.layer1.0.conv3.weight tensor(7.9817)
model.layer1.1.conv1.weight tensor(6.4255)
model.layer1.1.conv2.depthwise.weight tensor(41.2015)
model.layer1.1.conv2.pointwise.weight tensor(13.8813)
model.layer1.1.conv3.weight tensor(12.8609)
model.layer1.2.conv1.weight tensor(4.3928)
model.layer1.2.conv2.depthwise.weight tensor(41.6695)
model.layer1.2.conv2.pointwise.weight tensor(12.1137)
model.layer1.2.conv3.weight tensor(13.0310)
model.layer1.3.downsampleLayer.0.weight tensor(2.3664)
model.layer1.3.conv1.weight tensor(3.0040)
model.layer1.3.conv2.depthwise.weight tensor(42.8127)
model.layer1.3.conv2.pointwise.weight tensor(7.0075)
model.layer1.3.conv3.weight tensor(9.

In [24]:
for param in model_quantized.parameters():
  print(param.data)

tensor([0.8655, 0.7361, 0.8006, 1.0242, 0.7658, 0.6364, 1.0379, 0.7593, 0.7646,
        0.6700, 0.7588, 1.4384, 0.9710, 0.7369, 1.1581, 1.1651, 0.7751, 0.8824,
        0.9550, 0.7369, 0.7274, 0.8453, 0.7739, 0.9301, 1.0968, 0.8355, 0.8560,
        0.8008, 0.8540, 0.7144, 1.1408, 0.9329, 0.6801, 0.7863, 0.8213, 0.7962,
        0.9002, 0.8689, 0.8588, 0.9091, 0.7888, 0.9679, 0.7526, 0.8606, 0.8660,
        0.8038, 0.8201, 1.2672, 0.9480, 0.9948, 0.7132, 0.7175, 0.7448, 0.8928,
        0.5768, 0.9039, 0.7057, 0.7374, 1.2882, 0.7602, 0.8754, 0.9800, 0.8229,
        1.0440])
tensor([-0.1178,  0.0290,  0.6420,  0.1258, -0.1959,  0.2846,  0.3261,  0.2123,
         0.3637, -0.1634, -0.0418, -0.1854,  0.2616,  0.1446, -0.2409, -0.3890,
        -0.3365,  0.1352,  0.1197,  0.3240, -0.0303,  0.3303,  0.2386, -0.3729,
        -0.1747, -0.0275, -0.2110,  0.2276,  0.4093, -0.0264, -0.2664,  0.0461,
         0.5751, -0.1433, -0.0526,  0.1674, -0.5702, -0.0703,  0.6281,  0.2322,
         0.0901,  0.168

In [177]:
print_size_of_model(model_quantized)

Size (MB): 3.522077


In [25]:
test_loss, test_acc = evaluate_model(model_quantized, loss_fn, 'cpu', test_loader)

print(f"Accuracy = {test_acc:.4f} & Loss = {test_loss:.4f}")

Accuracy = 0.9264 & Loss = 17.6342


In [27]:
from prettytable import PrettyTable

def count_parameters(model_quantized):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model_quantized.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

count_parameters(model_quantized)

pytorch_total_params = sum(p.numel() for p in model_quantized.parameters())
print("Total Params: ", pytorch_total_params)


+------------------------------------------+------------+
|                 Modules                  | Parameters |
+------------------------------------------+------------+
|             model.bn1.weight             |     64     |
|              model.bn1.bias              |     64     |
| model.layer3.22.downsampleLayer.1.weight |     86     |
|  model.layer3.22.downsampleLayer.1.bias  |     86     |
+------------------------------------------+------------+
Total Trainable Params: 300
Total Params:  300


In [30]:
save_dir = pathlib.Path('/content/drive/MyDrive/Colab_Notebooks/MGU/Resnet_CIFAR10_Optimized')

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

optimized_model_name='resnet.ptl'

path_for_model = os.path.join(save_dir, optimized_model_name)

In [31]:
from torch.utils.mobile_optimizer import optimize_for_mobile

example = torch.rand(1, 3, 32, 32)

scripted_model = torch.jit.trace(model_quantized, example)

optimized_model = optimize_for_mobile(scripted_model)

optimized_model._save_for_lite_interpreter(path_for_model)

In [32]:
test_loss, test_acc = evaluate_model(scripted_model, loss_fn, 'cpu', test_loader)

print(f"Accuracy = {test_acc:.4f} & Loss = {test_loss:.4f}")

Accuracy = 0.9264 & Loss = 17.6342


In [37]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

count_parameters(model_to_quant_prepared)

pytorch_total_params = sum(p.numel() for p in model_to_quant_prepared.parameters())
print("Total Params: ", pytorch_total_params)


+------------------------------------------+------------+
|                 Modules                  | Parameters |
+------------------------------------------+------------+
|       model.conv1.depthwise.weight       |     27     |
|       model.conv1.pointwise.weight       |    192     |
|            model.bn1.0.weight            |     64     |
|             model.bn1.0.bias             |     64     |
| model.layer1.0.downsampleLayer.0.weight  |    640     |
|  model.layer1.0.downsampleLayer.0.bias   |     10     |
|       model.layer1.0.conv1.weight        |    4096    |
|        model.layer1.0.conv1.bias         |     64     |
|  model.layer1.0.conv2.depthwise.weight   |    576     |
|  model.layer1.0.conv2.pointwise.weight   |    4096    |
|   model.layer1.0.conv2.pointwise.bias    |     64     |
|       model.layer1.0.conv3.weight        |    640     |
|        model.layer1.0.conv3.bias         |     10     |
|       model.layer1.1.conv1.weight        |    640     |
|        model

In [ ]:
from fvcore.nn import FlopCountAnalysis
from fvcore.nn import flop_count_table

input =  torch.rand(1, 3, 32, 32, device='cpu')

input.cpu()

flops = FlopCountAnalysis(model_quantized.cpu(), input)
print(flop_count_table(flops))
print("Total number of FLOPS: ", flops.total())

# Pruning

In [25]:
save_dir = pathlib.Path('/content/drive/MyDrive/Colab_Notebooks/MGU/Resnet_CIFAR10_Pruning')

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

pruned_model_to_quant = QuantizedResNet(model)

num_iterations = 20

_best_sparsity = 0
_best_num_zeros = 0
_best_num_elements = 0
_best_acc = 0
best_pruned_model = copy.deepcopy(pruned_model)
_best_model_sparsity = 0

for i in range(num_iterations):

    print("\nPruning and Finetuning {}/{}".format(i + 1, num_iterations))

    parameters_to_prune = [
        (module, "weight") for module in filter(lambda m: type(m) == torch.nn.Conv2d, pruned_model_to_quant.modules())
    ]

        
    prune.global_unstructured(
        parameters_to_prune,
        pruning_method = prune.L1Unstructured,
        amount = 0.2,
    )

    num_zeros, num_elements, sparsity = measure_global_sparsity(pruned_model_to_quant, weight = True,
            bias = False, conv2d_use_mask = True,
            linear_use_mask = False)
    prefine_tunning_test_loss, prefine_tunning_test_acc = evaluate_model(pruned_model_to_quant, loss_fn, device, test_loader)

    print(f"Global sparsity = {sparsity * 100:.3f}% & test_accuracy = {prefine_tunning_test_acc:.4f} & params = {num_elements - num_zeros}")

    pruned_model_to_quant = fine_tune_model(
            model = pruned_model_to_quant, train_loader = train_loader,
            test_loader = test_loader, device = device,
            loss_fn = loss_fn, optimizer = optimizer,
            num_epochs = 20)
    
    model_name = 'resnet_pruned_iter%s.h5' % i
    torch.save(pruned_model_to_quant.state_dict(), os.path.join(save_dir, model_name))
    
    num_zeros, num_elements, sparsity = measure_global_sparsity(pruned_model_to_quant, weight = True,
            bias = False, conv2d_use_mask = True,
            linear_use_mask = False)
    test_loss, test_acc = evaluate_model(pruned_model_to_quant, loss_fn, device, test_loader)

    if test_acc > _best_acc:
        _best_sparsity = sparsity
        _best_num_zeros = num_zeros
        _best_num_elements = num_elements
        _best_acc = test_acc
        best_pruned_model = copy.deepcopy(pruned_model_to_quant)

    print(f"Post fine-tuning: Global sparsity = {sparsity * 100:.3f}% & test_accuracy = {test_acc:.4f} & params = {num_elements - num_zeros}")




Pruning and Finetuning 1/20
Global sparsity = 18.338% & test_accuracy = 0.9278 & params = 2626239
epoch = 0 train loss = 2.360, test_loss = 17.383, test_accuracy = 0.9272
epoch = 1 train loss = 2.184, test_loss = 17.523, test_accuracy = 0.9268
epoch = 2 train loss = 1.952, test_loss = 17.601, test_accuracy = 0.9274
epoch = 3 train loss = 2.212, test_loss = 17.501, test_accuracy = 0.9277
epoch = 4 train loss = 2.374, test_loss = 17.661, test_accuracy = 0.9269
epoch = 5 train loss = 2.306, test_loss = 17.533, test_accuracy = 0.9273
epoch = 6 train loss = 2.150, test_loss = 17.324, test_accuracy = 0.9275
epoch = 7 train loss = 2.093, test_loss = 17.499, test_accuracy = 0.9273
epoch = 8 train loss = 2.356, test_loss = 17.473, test_accuracy = 0.9265
epoch = 9 train loss = 1.948, test_loss = 17.498, test_accuracy = 0.9271
epoch = 10 train loss = 2.267, test_loss = 17.583, test_accuracy = 0.9276
epoch = 11 train loss = 2.003, test_loss = 17.468, test_accuracy = 0.9279
epoch = 12 train loss =

KeyboardInterrupt: ignored

In [26]:
save_dir = pathlib.Path('/content/drive/MyDrive/Colab_Notebooks/MGU/Resnet_CIFAR10_Pruning')

device = 'cuda'
pruned_loaded_model = QuantizedResNet(model)
parameters_to_prune = [
    (module, "weight") for module in filter(lambda m: type(m) == torch.nn.Conv2d, pruned_loaded_model.modules())
]

    
prune.global_unstructured(
    parameters_to_prune,
    pruning_method = prune.L1Unstructured,
    amount = 0,
)
i = 0
for file in os.listdir(save_dir):
    
    pruned_loaded_model.load_state_dict(torch.load(os.path.join(save_dir, file)))
    test_loss, test_acc = evaluate_model(pruned_loaded_model, loss_fn, device, test_loader)
    num_zeros, num_elements, sparsity = measure_global_sparsity(pruned_model_to_quant, weight = True,
            bias = False, conv2d_use_mask = True,
            linear_use_mask = False)
    print(f"{i}: Global sparsity = {sparsity * 100:.3f}% & test_accuracy = {test_acc:.4f} & params = {num_elements - num_zeros}")
    i += 1

0: Global sparsity = 18.338% & test_accuracy = 0.9269 & params = 2626239
1: Global sparsity = 33.008% & test_accuracy = 0.9257 & params = 2154444
2: Global sparsity = 44.744% & test_accuracy = 0.9236 & params = 1777008
3: Global sparsity = 54.133% & test_accuracy = 0.9182 & params = 1475059
4: Global sparsity = 61.645% & test_accuracy = 0.9118 & params = 1233500
5: Global sparsity = 67.654% & test_accuracy = 0.8970 & params = 1040253
6: Global sparsity = 72.461% & test_accuracy = 0.8636 & params = 885655
7: Global sparsity = 76.307% & test_accuracy = 0.8094 & params = 761977
8: Global sparsity = 79.383% & test_accuracy = 0.7194 & params = 663034
9: Global sparsity = 81.844% & test_accuracy = 0.5941 & params = 583880
10: Global sparsity = 83.813% & test_accuracy = 0.4839 & params = 520557


In [48]:
pruned_best_model = copy.deepcopy(pruned_loaded_model)

In [54]:
device = 'cuda'
save_dir = pathlib.Path('/content/drive/MyDrive/Colab_Notebooks/MGU/Resnet_CIFAR10_Pruning')
model_name = 'resnet_pruned_iter4.h5'

pruned_best_model = QuantizedResNet(model)
parameters_to_prune = [
    (module, "weight") for module in filter(lambda m: type(m) == torch.nn.Conv2d, pruned_best_model.modules())
]
 
prune.global_unstructured(
    parameters_to_prune,
    pruning_method = prune.L1Unstructured,
    amount = 0,
)

pruned_best_model.load_state_dict(torch.load(os.path.join(save_dir, model_name)))

test_loss, test_acc = evaluate_model(pruned_best_model, loss_fn, device, test_loader)
num_zeros, num_elements, sparsity = measure_global_sparsity(pruned_best_model, weight = True,
        bias = False, conv2d_use_mask = True,
        linear_use_mask = False)
print(f"Global sparsity = {sparsity * 100:.3f}% & test_accuracy = {test_acc:.4f}")# & params = {num_elements - num_zeros}")

pruned_best_model = remove_parameters(model = pruned_best_model)

Global sparsity = 61.645% & test_accuracy = 0.9118


In [55]:
save_dir = pathlib.Path('/content/drive/MyDrive/Colab_Notebooks/MGU/Resnet_CIFAR10_Best_Pruned')

model_name = 'pruned_resnet.h5'

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)



torch.save(pruned_best_model.state_dict(), os.path.join(save_dir, model_name))

In [31]:
count_parameters(pruned_best_model)

pytorch_total_params = sum(p.numel() for p in pruned_best_model.parameters())
print("Total Params: ", pytorch_total_params)


+------------------------------------------+------------+
|                 Modules                  | Parameters |
+------------------------------------------+------------+
|       model.conv1.depthwise.weight       |     27     |
|       model.conv1.pointwise.weight       |    192     |
|             model.bn1.weight             |     64     |
|              model.bn1.bias              |     64     |
| model.layer1.0.downsampleLayer.0.weight  |    640     |
| model.layer1.0.downsampleLayer.1.weight  |     10     |
|  model.layer1.0.downsampleLayer.1.bias   |     10     |
|       model.layer1.0.conv1.weight        |    4096    |
|        model.layer1.0.bn1.weight         |     64     |
|         model.layer1.0.bn1.bias          |     64     |
|  model.layer1.0.conv2.depthwise.weight   |    576     |
|  model.layer1.0.conv2.pointwise.weight   |    4096    |
|        model.layer1.0.bn2.weight         |     64     |
|         model.layer1.0.bn2.bias          |     64     |
|       model.

In [32]:
from fvcore.nn import FlopCountAnalysis
from fvcore.nn import flop_count_table

input =  torch.rand(1, 3, 32, 32, device='cpu')

input.cpu()

flops = FlopCountAnalysis(pruned_best_model.cpu(), input)
print(flop_count_table(flops))
print("Total number of FLOPS: ", flops.total())

| module                       | #parameters or shape   | #flops     |
|:-----------------------------|:-----------------------|:-----------|
| model                        | 3.251M                 | 0.189G     |
|  conv1                       |  0.219K                |  0.224M    |
|   conv1.depthwise            |   27                   |   27.648K  |
|    conv1.depthwise.weight    |    (3, 1, 3, 3)        |            |
|   conv1.pointwise            |   0.192K               |   0.197M   |
|    conv1.pointwise.weight    |    (64, 3, 1, 1)       |            |
|  bn1                         |  0.128K                |  0.131M    |
|   bn1.weight                 |   (64,)                |            |
|   bn1.bias                   |   (64,)                |            |
|  layer1                      |  30.084K               |  25.802M   |
|   layer1.0                   |   10.344K              |   10.592M  |
|    layer1.0.downsampleLayer  |    0.66K               |    0.676M  |
|    l

# Quantization of pruned model

In [56]:
modules_to_fuse = [
    ['model.bn1', 'model.relu'],

    #layer1
    ['model.layer1.0.downsampleLayer.0', 'model.layer1.0.downsampleLayer.1'],
    ['model.layer1.0.conv1', 'model.layer1.0.bn1'],
    ['model.layer1.0.conv2.pointwise', 'model.layer1.0.bn2'],
    ['model.layer1.0.conv3', 'model.layer1.0.bn3'],

    ['model.layer1.1.conv1', 'model.layer1.1.bn1'],
    ['model.layer1.1.conv2.pointwise', 'model.layer1.1.bn2'],
    ['model.layer1.1.conv3', 'model.layer1.1.bn3'],

    ['model.layer1.2.conv1', 'model.layer1.2.bn1'],
    ['model.layer1.2.conv2.pointwise', 'model.layer1.2.bn2'],
    ['model.layer1.2.conv3', 'model.layer1.2.bn3'],

    ['model.layer1.3.downsampleLayer.0', 'model.layer1.3.downsampleLayer.1'],
    ['model.layer1.3.conv1', 'model.layer1.3.bn1'],
    ['model.layer1.3.conv2.pointwise', 'model.layer1.3.bn2'],
    ['model.layer1.3.conv3', 'model.layer1.3.bn3'],

    #layer2
    ['model.layer2.0.conv1', 'model.layer2.0.bn1'],
    ['model.layer2.0.conv2.pointwise', 'model.layer2.0.bn2'],
    ['model.layer2.0.conv3', 'model.layer2.0.bn3'],

    ['model.layer2.1.conv1', 'model.layer2.1.bn1'],
    ['model.layer2.1.conv2.pointwise', 'model.layer2.1.bn2'],
    ['model.layer2.1.conv3', 'model.layer2.1.bn3'],

    ['model.layer2.2.conv1', 'model.layer2.2.bn1'],
    ['model.layer2.2.conv2.pointwise', 'model.layer2.2.bn2'],
    ['model.layer2.2.conv3', 'model.layer2.2.bn3'],

    ['model.layer2.3.downsampleLayer.0', 'model.layer2.3.downsampleLayer.1'],
    ['model.layer2.3.conv1', 'model.layer2.3.bn1'],
    ['model.layer2.3.conv2.pointwise', 'model.layer2.3.bn2'],
    ['model.layer2.3.conv3', 'model.layer2.3.bn3'],

    #layer3
    ['model.layer3.0.conv1', 'model.layer3.0.bn1'],
    ['model.layer3.0.conv2.pointwise', 'model.layer3.0.bn2'],
    ['model.layer3.0.conv3', 'model.layer3.0.bn3'],

    ['model.layer3.1.conv1', 'model.layer3.1.bn1'],
    ['model.layer3.1.conv2.pointwise', 'model.layer3.1.bn2'],
    ['model.layer3.1.conv3', 'model.layer3.1.bn3'],

    ['model.layer3.2.conv1', 'model.layer3.2.bn1'],
    ['model.layer3.2.conv2.pointwise', 'model.layer3.2.bn2'],
    ['model.layer3.2.conv3', 'model.layer3.2.bn3'],

    ['model.layer3.3.conv1', 'model.layer3.3.bn1'],
    ['model.layer3.3.conv2.pointwise', 'model.layer3.3.bn2'],
    ['model.layer3.3.conv3', 'model.layer3.3.bn3'],

    ['model.layer3.4.conv1', 'model.layer3.4.bn1'],
    ['model.layer3.4.conv2.pointwise', 'model.layer3.4.bn2'],
    ['model.layer3.4.conv3', 'model.layer3.4.bn3'],

    ['model.layer3.5.conv1', 'model.layer3.5.bn1'],
    ['model.layer3.5.conv2.pointwise', 'model.layer3.5.bn2'],
    ['model.layer3.5.conv3', 'model.layer3.5.bn3'],

    ['model.layer3.6.conv1', 'model.layer3.6.bn1'],
    ['model.layer3.6.conv2.pointwise', 'model.layer3.6.bn2'],
    ['model.layer3.6.conv3', 'model.layer3.6.bn3'],

    ['model.layer3.7.conv1', 'model.layer3.7.bn1'],
    ['model.layer3.7.conv2.pointwise', 'model.layer3.7.bn2'],
    ['model.layer3.7.conv3', 'model.layer3.7.bn3'],

    ['model.layer3.8.conv1', 'model.layer3.8.bn1'],
    ['model.layer3.8.conv2.pointwise', 'model.layer3.8.bn2'],
    ['model.layer3.8.conv3', 'model.layer3.8.bn3'],

    ['model.layer3.9.conv1', 'model.layer3.9.bn1'],
    ['model.layer3.9.conv2.pointwise', 'model.layer3.9.bn2'],
    ['model.layer3.9.conv3', 'model.layer3.9.bn3'],

    ['model.layer3.10.conv1', 'model.layer3.10.bn1'],
    ['model.layer3.10.conv2.pointwise', 'model.layer3.10.bn2'],
    ['model.layer3.10.conv3', 'model.layer3.10.bn3'],

    ['model.layer3.11.conv1', 'model.layer3.11.bn1'],
    ['model.layer3.11.conv2.pointwise', 'model.layer3.11.bn2'],
    ['model.layer3.11.conv3', 'model.layer3.11.bn3'],

    ['model.layer3.12.conv1', 'model.layer3.12.bn1'],
    ['model.layer3.12.conv2.pointwise', 'model.layer3.12.bn2'],
    ['model.layer3.12.conv3', 'model.layer3.12.bn3'],

    ['model.layer3.13.conv1', 'model.layer3.13.bn1'],
    ['model.layer3.13.conv2.pointwise', 'model.layer3.13.bn2'],
    ['model.layer3.13.conv3', 'model.layer3.13.bn3'],

    ['model.layer3.14.conv1', 'model.layer3.14.bn1'],
    ['model.layer3.14.conv2.pointwise', 'model.layer3.14.bn2'],
    ['model.layer3.14.conv3', 'model.layer3.14.bn3'],

    ['model.layer3.15.conv1', 'model.layer3.15.bn1'],
    ['model.layer3.15.conv2.pointwise', 'model.layer3.15.bn2'],
    ['model.layer3.15.conv3', 'model.layer3.15.bn3'],

    ['model.layer3.16.conv1', 'model.layer3.16.bn1'],
    ['model.layer3.16.conv2.pointwise', 'model.layer3.16.bn2'],
    ['model.layer3.16.conv3', 'model.layer3.16.bn3'],

    ['model.layer3.17.conv1', 'model.layer3.17.bn1'],
    ['model.layer3.17.conv2.pointwise', 'model.layer3.17.bn2'],
    ['model.layer3.17.conv3', 'model.layer3.17.bn3'],

    ['model.layer3.18.conv1', 'model.layer3.18.bn1'],
    ['model.layer3.18.conv2.pointwise', 'model.layer3.18.bn2'],
    ['model.layer3.18.conv3', 'model.layer3.18.bn3'],

    ['model.layer3.19.conv1', 'model.layer3.19.bn1'],
    ['model.layer3.19.conv2.pointwise', 'model.layer3.19.bn2'],
    ['model.layer3.19.conv3', 'model.layer3.19.bn3'],

    ['model.layer3.20.conv1', 'model.layer3.20.bn1'],
    ['model.layer3.20.conv2.pointwise', 'model.layer3.20.bn2'],
    ['model.layer3.20.conv3', 'model.layer3.20.bn3'],

    ['model.layer3.21.conv1', 'model.layer3.21.bn1'],
    ['model.layer3.21.conv2.pointwise', 'model.layer3.21.bn2'],
    ['model.layer3.21.conv3', 'model.layer3.21.bn3'],

    ['model.layer3.22.conv1', 'model.layer3.22.bn1'],
    ['model.layer3.22.conv2.pointwise', 'model.layer3.22.bn2'],
    ['model.layer3.22.conv3', 'model.layer3.22.bn3'],

    #layer4
    ['model.layer4.0.conv1', 'model.layer4.0.bn1'],
    ['model.layer4.0.conv2.pointwise', 'model.layer4.0.bn2'],
    ['model.layer4.0.conv3', 'model.layer4.0.bn3'],

    ['model.layer4.1.conv1', 'model.layer4.1.bn1'],
    ['model.layer4.1.conv2.pointwise', 'model.layer4.1.bn2'],
    ['model.layer4.1.conv3', 'model.layer4.1.bn3'],

    ['model.layer4.2', 'model.layer4.3'],
]



In [57]:
def evaluate(model, loss_fn, data_loader, neval_batches):
    model.eval()
    cnt = 0
    with torch.no_grad():
        for image, target in data_loader:
            output = model(image)
            loss = loss_fn(output, target)
            cnt += 1
            test_loss, test_acc = evaluate_model(model, loss_fn, 'cpu', test_loader)
            print(f"Batch num = {cnt} & Accuracy = {test_acc:.4f} & Loss = {test_loss:.4f}")
            if cnt >= neval_batches:
                 return test_loss, test_acc

    return test_loss, test_acc

In [58]:
quant_pruned_best_model = copy.deepcopy(pruned_best_model)

In [59]:
num_calibration_batches = 5

quant_pruned_best_model.to('cpu')
quant_pruned_best_model.eval()

quantization_config = torch.quantization.get_default_qconfig("qnnpack")
quant_pruned_best_model.qconfig = quantization_config

quant_pruned_best_model_fused = torch.quantization.fuse_modules(quant_pruned_best_model, modules_to_fuse)

quant_pruned_best_model_prepared = torch.quantization.prepare(quant_pruned_best_model_fused, inplace=False)

test_loss, test_acc = evaluate(quant_pruned_best_model_prepared, loss_fn, test_loader, neval_batches=num_calibration_batches)
print(f"Tunned: Accuracy = {test_acc:.4f} & Loss = {test_loss:.4f}")

print(quant_pruned_best_model_prepared.model.layer1[0].conv1)
best_model_pruned_and_quantized = torch.quantization.convert(quant_pruned_best_model_prepared, inplace=False)

Batch num = 1 & Accuracy = 0.9118 & Loss = 20.7982
Batch num = 2 & Accuracy = 0.9118 & Loss = 20.7982
Batch num = 3 & Accuracy = 0.9118 & Loss = 20.7982
Batch num = 4 & Accuracy = 0.9118 & Loss = 20.7982
Batch num = 5 & Accuracy = 0.9118 & Loss = 20.7982
Tunned: Accuracy = 0.9118 & Loss = 20.7982
Conv2d(
  64, 64, kernel_size=(1, 1), stride=(1, 1)
  (activation_post_process): HistogramObserver(min_val=-7.405702590942383, max_val=8.173539161682129)
)


In [60]:
print_size_of_model(best_model_pruned_and_quantized)

Size (MB): 3.481819


In [61]:
model_name = 'best_pruned_quantized_resnet101.h5'
save_dir = pathlib.Path('/content/drive/MyDrive/Colab_Notebooks/MGU/Resnet_CIFAR10_Best_Pruned_Quantized')

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

torch.save(best_model_pruned_and_quantized.state_dict(), os.path.join(save_dir, model_name))

In [62]:
test_loss, test_acc = evaluate_model(best_model_pruned_and_quantized, loss_fn, 'cpu', test_loader)

print(f"Accuracy = {test_acc:.4f} & Loss = {test_loss:.4f}")

Accuracy = 0.9122 & Loss = 20.4990


In [63]:
save_dir = pathlib.Path('/content/drive/MyDrive/Colab_Notebooks/MGU/Resnet_CIFAR10_Optimized')

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

path_for_pruned_model='best_resnet.ptl'

path_for_model = os.path.join(save_dir, path_for_pruned_model)

In [64]:
from torch.utils.mobile_optimizer import optimize_for_mobile

example = torch.rand(1, 3, 32, 32)

scripted_best_model_pruned = torch.jit.trace(best_model_pruned_and_quantized, example)

optimized_best_model_pruned = optimize_for_mobile(scripted_best_model_pruned)

optimized_best_model_pruned._save_for_lite_interpreter(path_for_pruned_model)